# 主程式遞迴Function
### 1.判斷子查詢相關SQL功能
- SELECT : 判斷ttype is DML + 值為 "SELECT"
- FROM : 判斷ttype is Keyword + 值為 "FROM"
- WHERE : 判斷instance is Where + 值含 "WHERE"

### 2. 紀錄上述state作為後續標註
- state ：初始SQL狀態。樣式為 # MAIN、SELECT、FROM、WHERE再持續往上加
- statement : 當前SQL狀態。樣式為 # MAIN、SELECT、FROM、WHERE再持續往上加
- count_dict：紀錄不同樣式的子查詢出現次數。(作為ID編碼 4.會用到)

### 3. 觀察是否有子查詢
- is_subselect : 檢查token是否有子查詢, 
 - 回傳 boolean 
- trace_token_is_subselect ： 有些功能語句會在Parenthesis外包裝一個Identifier，需要再往下追蹤一層。
 - 回傳dictionary，KEY : 標註當前SQL狀態的次數、VALUE:Token訊息。
 
### 4. 將回傳的Token Value依照標註儲存於DICT
- 儲存KEY - new_state：state + "_" + state + (count_dict[statement])
- 儲存Value - token.value
- 儲存的字典檔：subquery_dict

### 5. 遞迴功能(將子查詢持續往下查找分成 1 class and 2nd class)
- 遞迴參數：
 - the_query = 儲存Value並去掉頭尾的「括號」(4.), 
 - state = new_state(4.), 
 - subquery_dict_params = 儲存的字典檔(4.)
- 遞迴結束條件：
 - sqiparse樹結構 1 層Trace完後，無任何有SELECT語句的子查詢。

In [9]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where
from sqlparse.tokens import Keyword, DML, DDL
import re

In [37]:
def split_subquery(the_query, state = "MAIN", subquery_dict_params = {}):
    # Function local variables and codes 
    # Record query's tokens
    subquery_dict = subquery_dict_params
    print(subquery_dict)
    res = sqlparse.parse(the_query)
    stmt = res[0]
    subquery_dict[state] = the_query
    print(stmt._pprint_tree())

    # 01 Record subquery in differnent statement out times
    count_dict = {"MAIN" : 0, "SELECT" : 0, "FROM" : 0, "WHERE" : 0}
    statement = ""
    for token in stmt.tokens:

        # 02 Setting state
        if token.ttype is DML and token.value.upper() == "SELECT":
            statement = "SELECT"
            print("SELECT表示句:" + token.value.upper())
        if token.ttype is Keyword and token.value.upper() == "FROM":
            statement = "FROM"
            print("FROM表示句:" + token.value.upper())
        if isinstance(token, Where) and bool(re.search("WHERE",token.value.upper())):
            statement = "WHERE"
            print("WHERE表示句:" + token.value.upper())

        # 03 is_subselect check 
        ## Record statement count and Print subselect position 
        ### 03_1 1st class trace
        ### Save subquerys and recursive
        if (is_subselect(token)):
            count_dict[statement] += 1
            the_subquery = token.value # Delete Captain and tail (parentheses)
            the_subtoken = token.tokens
            new_state = state + "_" + statement + str(count_dict[statement]) # Subselect key set new state
            subquery_dict[new_state] =  the_subquery
            split_subquery(the_query = the_subquery[1:-1], 
                           state = new_state, 
                           subquery_dict_params = subquery_dict)
            continue ## 1st class show do not need go to 2nd class

        ### 03_2 2nd class trace    
        ### Save subquerys and recursive 
        subselect_dict = {}
        subselect_dict = trace_token_is_subselect(token, count_dict, statement) 
        for sub_item in subselect_dict.items():
            the_subquery = sub_item[1].value # Delete Captain and tail (parentheses)
            the_subtokens = sub_item[1]
            new_state = state + "_" + statement + str(sub_item[0]) # Subselect key set new state
            subquery_dict[new_state] = the_subquery
            print(new_state)
            print(the_subquery)
            ### recirsive subselect and Update the subquery_dict
            split_subquery(the_query = the_subquery[1:-1], 
                           state = new_state, 
                           subquery_dict_params = subquery_dict)
  
        
    return(subquery_dict)

## Function : 是否有子查詢

In [2]:
def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

## Function : 追蹤token內的列表並確認是否有子查

In [3]:
def trace_token_is_subselect(parsed, count_dict, state):
    subselect_dict = {} # count_dict as KEY, item as VALUE
    
    ## No group return null subquery_dict
    if not parsed.is_group:
        return subselect_dict
    
    ## With group return Record subquery_dict
    for item in parsed.tokens:
        if is_subselect(item):
            count_dict[state] += 1
            subselect_dict[ count_dict[state] ] = item
    
    return subselect_dict

In [4]:
def __main__():
    the_query = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1
               limit 15'''
    split_subquery( the_query ,
                  state = "MAIN",
                  subquery_dict_params = {})

In [40]:
the_query = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1---
               limit 15'''
sql = """
    select K.a,K.b from (select H.b from (select G.c from (select F.d from
    (select E.e from MEOW.A as t1 , MEOW.B, MEOW.C, MEOW.D, MEOW.E), F), G), H), I, J, K order by 1,2;
    """

split_subquery( the_query ,
                  state = "MAIN",
                  subquery_dict_params = {})

{}
|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Keyword 'FINAL'
|- 5 Whitespace ' '
|- 6 Keyword 'as'
|- 7 Whitespace ' '
|- 8 Newline ' '
|- 9 Whitespace ' '
|- 10 Whitespace ' '
|- 11 Whitespace ' '
|- 12 Whitespace ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 Whitespace ' '
|- 17 Whitespace ' '
|- 18 Whitespace ' '
|- 19 Whitespace ' '
|- 20 Whitespace ' '
|- 21 Whitespace ' '
|- 22 Whitespace ' '
|- 23 Whitespace ' '
|- 24 DML 'select'
|- 25 Whitespace ' '
|- 26 IdentifierList 'a1, co...'
|  |- 0 Identifier 'a1'
|  |  `- 0 Name 'a1'
|  |- 1 Punctuation ','
|  |- 2 Whitespace ' '
|  `- 3 Identifier 'count'
|     `- 0 Name 'count'
|- 27 Parenthesis '(a2)'
|  |- 0 Punctuation '('
|  |- 1 Identifier 'a2'
|  |  `- 0 Name 'a2'
|  `- 2 Punctuation ')'
|- 28 Whitespace ' '
|- 29 Newline ' '
|- 30 Whitespace ' '
|- 31 Whitespace ' '
|- 32 Whitespace ' '
|- 33 Whitespace ' '
|- 34 Whitespace ' '
|- 35 Whitespace ' '
|- 36 Whi

{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1---\n               limit 15",
 'MAIN_FROM1': 'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ',
 'MAIN_FROM1_FROM1': 'SELECT * FROM MEOW.CCC_table ',
 'MAIN_WHERE1': 'SELECT distinct(b1) FROM MEOW.BBB_table '}